<a href="https://colab.research.google.com/github/PacktPublishing/Hands-On-Computer-Vision-with-Detectron2/blob/main/Chapter12/Detectron2_Chapter12_Scripting.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 12 - Tracing versus Scripting

## Creating a model

In [ ]:
import torch
import torch.nn as nn

device = "cuda"
if not torch.cuda.is_available():
  device = "cpu"

class SimplePyTorchModel(nn.Module):
  def __init__(self):
    super(SimplePyTorchModel, self).__init__()
    self.linear = nn.Linear(4, 1)   
    self.linear.weight.data.fill_(0.01)
    self.linear.bias.data.fill_(0.01)

  def forward(self, x):
    y = self.linear(x)
    if y.sum() > 0:
      return y
    else:
      return -y

pt_model = SimplePyTorchModel()
pt_model.to(device)
print(pt_model)

SimplePyTorchModel(
  (linear): Linear(in_features=4, out_features=1, bias=True)
)


In [ ]:
for p in pt_model.parameters():
  print(p.data)

tensor([[0.0100, 0.0100, 0.0100, 0.0100]], device='cuda:0')
tensor([0.0100], device='cuda:0')


## Perform Predictions

In [ ]:
X = torch.tensor(
    data  = [[1, 2, 3, 4]], 
    dtype = torch.float32)
X = X.to(device)
with torch.no_grad():
  y = pt_model(X)
  print(y)

tensor([[0.1100]], device='cuda:0')


## Save the model

In [ ]:
pt_model.to("cpu")

SimplePyTorchModel(
  (linear): Linear(in_features=4, out_features=1, bias=True)
)

In [ ]:
torch.save(pt_model,"pt_model.pt")

## Exporting to TorchScript using Tracing

In [ ]:
dummy_X = torch.tensor(
    data  = [[-1, -2, -3, -4]], 
    dtype = torch.float32)
traced_model = torch.jit.trace(
    pt_model, 
    example_inputs = dummy_X)

<ipython-input-1-1b0e76ddf8bf>:17: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if y.sum() > 0:


In [ ]:
traced_model.to(device)
for p in traced_model.parameters():
  print(p.data)

tensor([[0.0100, 0.0100, 0.0100, 0.0100]], device='cuda:0')
tensor([0.0100], device='cuda:0')


In [ ]:
print(traced_model.code)

def forward(self,
    x: Tensor) -> Tensor:
  linear = self.linear
  return torch.neg((linear).forward(x, ))



## Test the exported model

In [ ]:
with torch.no_grad():
  y = traced_model(X)
  print(y)

tensor([[-0.1100]], device='cuda:0')


## Export to TorchScript using Scripting

In [ ]:
scripted_model = torch.jit.script(pt_model)
print(scripted_model.code)

def forward(self,
    x: Tensor) -> Tensor:
  linear = self.linear
  y = (linear).forward(x, )
  if bool(torch.gt(torch.sum(y), 0)):
    _0 = y
  else:
    _0 = torch.neg(y)
  return _0



In [ ]:
for p in scripted_model.parameters():
  print(p.data)

tensor([[0.0100, 0.0100, 0.0100, 0.0100]], device='cuda:0')
tensor([0.0100], device='cuda:0')


In [ ]:
scripted_model.to(device)
with torch.no_grad():
  y = scripted_model(X)
  print(y)

tensor([[0.1100]], device='cuda:0')


## Save the model

In [ ]:
scripted_model.to("cpu").save("scripted_model.pt")

## Test loading

In [ ]:
loaded_model = torch.jit.load("scripted_model.pt")

In [ ]:
for p in loaded_model.parameters():
  print(p)

tensor([[0.0100, 0.0100, 0.0100, 0.0100]], requires_grad=True)
tensor([0.0100], requires_grad=True)


In [ ]:
loaded_model.to(device)

RecursiveScriptModule(
  original_name=SimplePyTorchModel
  (linear): RecursiveScriptModule(original_name=Linear)
)

In [ ]:
loaded_model(X)

tensor([[0.1100]], device='cuda:0', grad_fn=<AddmmBackward0>)

## Download model

In [ ]:
from google.colab import files
files.download('scripted_model.pt')
files.download('pt_model.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>